<a href="https://www.kaggle.com/code/rubanzasilva/symptom-disease?scriptVersionId=227565567" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction: Predictive model for differential diagnosis

In this notebook, our goal is to develop a model that can take in a patient's symptoms as an input and return a list of the top 3 possible classes (diseases) alongside confidence values for each class expressed as probabilities.


## Library and Data import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/symptom2disease/Symptom2Disease.csv
/kaggle/input/medical-corpus/dx_datav1.csv.csv


In [2]:
%%time
%pip install catboost
%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
%pip install --upgrade pip
%pip install tqdm
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 991 ms, sys: 277 ms, total: 1.27 s
Wall time: 1min 23s


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm

#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)
from fastai.text.all import *


from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold, cross_val_score,train_test_split,GridSearchCV


from pathlib import Path
import os

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv


import warnings


#from openfe import OpenFE, transform
#from autogluon.tabular import TabularDataset, TabularPredictor

#import h2o
#from h2o.automl import H2OAutoML

import gc

import optuna
from optuna.samplers import TPESampler

import pickle
from joblib import dump, load

#import sweetviz as sv
#from IPython.display import FileLink

#import h2o
#from h2o.automl import H2OAutoML

Possible Approaches

1. Using the fastai text transfer learning approach which employs a language model (A model trained to predict the next text in a sentence).
   This employs self-supervised learning, to quote the fastai book
   > Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.
3. Using traditional tabular models and implementations such as xgboost, to do this we would have to convert our text values (symptoms) to numerical features.

In [4]:
path = Path('/kaggle/input/symptom2disease/')
path

Path('/kaggle/input/symptom2disease')

# Exploratory Data Analysis

In [5]:
!ls /kaggle/input/symptom2disease

Symptom2Disease.csv


In [6]:
df = pd.read_csv(path/'Symptom2Disease.csv',index_col=0)
df.head()

,label,text
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [37]:
df

,label,text
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."
...,...,...
295,diabetes,"I'm shaking and trembling all over. I've lost my sense of taste and smell, and I'm exhausted. I occasionally get palpitations or a speeding heart."
296,diabetes,"Particularly in the crevices of my skin, I have skin rashes and irritations. My skin bruises and cuts take a while to heal as well."
297,diabetes,I regularly experience these intense urges and the want to urinate. I frequently feel drowsy and lost. I've also significantly lost my vision.
298,diabetes,"I have trouble breathing, especially outside. I start to feel hot and start to sweat. I frequently have urinary tract infections and yeast infections."


In [7]:
df.shape

(1200, 2)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 299
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 28.1+ KB


In [40]:
#Missing values returned as percentages.
missing_percentages = df.isnull().mean() * 100
sorted_missing_percentages = missing_percentages.sort_values(ascending=False)
print(sorted_missing_percentages)

label    0.0
text     0.0
dtype: float64


In [10]:
df.describe().T

,count,unique,top,freq
label,1200,24,Psoriasis,50
text,1200,1153,"I've been feeling extremely scratchy, sick, and worn out. In addition, I've lost weight and have a temperature. My urine is dark, and my skin has turned yellow. Additionally, I have been experiencing stomach pain.",4


In [38]:
df['label'].nunique()

24

In [12]:
df['label'].unique()

array(['Psoriasis', 'Varicose Veins', 'Typhoid', 'Chicken pox', 'Impetigo', 'Dengue', 'Fungal infection', 'Common Cold',
       'Pneumonia', 'Dimorphic Hemorrhoids', 'Arthritis', 'Acne', 'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Jaundice', 'Malaria', 'urinary tract infection', 'allergy', 'gastroesophageal reflux disease',
       'drug reaction', 'peptic ulcer disease', 'diabetes'], dtype=object)

In [14]:
df['label'].value_counts()

label
Psoriasis                          50
Varicose Veins                     50
Typhoid                            50
Chicken pox                        50
Impetigo                           50
Dengue                             50
Fungal infection                   50
Common Cold                        50
Pneumonia                          50
Dimorphic Hemorrhoids              50
Arthritis                          50
Acne                               50
Bronchial Asthma                   50
Hypertension                       50
Migraine                           50
Cervical spondylosis               50
Jaundice                           50
Malaria                            50
urinary tract infection            50
allergy                            50
gastroesophageal reflux disease    50
drug reaction                      50
peptic ulcer disease               50
diabetes                           50
Name: count, dtype: int64

So there are 24 diseases in this dataset

In [ ]:
#X = df['label']
#y = df['text']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(
    #X, y, test_size=0.20, random_state=42)

In [ ]:
#splits = RandomSplitter(valid_pct=0.2)(range_of(df))

# Approach 1 - Text transfer learning
Train a text classifier from a pretrained model


## Using the high-level API

In [15]:
dls = TextDataLoaders.from_df(df, path=path, text_col='text', label_col='label',valid_pct=0.2)

In [16]:
dls.show_batch()

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma
2,"xxbos xxunk xxunk that xxmaj i 've been experiencing shortness of breath , a persistent cough , and a high fever . xxmaj i 've also been producing a lot of thick , mucoid sputum , and xxmaj i 'm feeling quite drained and exhausted . xxmaj it 's all been rather tough to deal with , and xxmaj i 'm feeling quite sick at the xxunk",Bronchial Asthma
3,"xxbos xxmaj i 've been feeling really ill lately . xxmaj i 've had this persistent cough and difficulty breathing , and my fever has been off the xxunk . xxmaj i 'm also feeling extremely exhausted , and xxmaj i 've been producing a lot of mucus when i cough . xxmaj it 's just been so overwhelming to deal with all of these symptoms",Bronchial Asthma
4,xxbos xxmaj my eyes are red and watery all the time . xxmaj i 've also had this pressure in my sinuses that wo n't go away . xxmaj i 'm always feeling tired and xxmaj i 've been having a lot of trouble breathing . xxmaj i 've also had a lot of gunk in my throat and my lymph nodes are swollen .,Common Cold
5,"xxbos xxmaj i 've been struggling with difficulty breathing , a constant cough , and fatigue . xxmaj my fever is extremely high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's been so hard to deal with all of this , and xxmaj i 'm just feeling so drained and worn out",Bronchial Asthma
6,xxbos xxmaj i 've been having a really hard time going to the bathroom lately . xxmaj it 's really painful and xxmaj i 'm only able to go every few days . xxmaj there 's also a lot of pain in my anus and around that area . xxmaj my stool has been really bloody and my anus feels really irritated .,Dimorphic Hemorrhoids
7,"xxbos xxmaj my eyes are always red and itchy , and my nose feels all stuffy and congested . i just feel kind of sick and tired all the time , and i keep coughing up all this gunk . xxmaj my throat feels sore and scratchy , and xxmaj i 've noticed that the bumps on my neck are xxunk than usual",Common Cold
8,"xxbos xxmaj xxunk , xxmaj mom and xxmaj xxunk xxmaj recently , xxmaj i 've felt incredibly worn out and have had a cough that wo n't go away . xxmaj my fever is really high , and my breathing has become shallow . xxmaj additionally , xxmaj i 've been coughing up a lot of mucous , thick sputum .",Bronchial Asthma


> the library automatically processed all the texts to split then in tokens, adding some special tokens like:

xxbos to indicate the beginning of a te
t
xxmaj to indicate the next word was capitalized

In [17]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [18]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.810592,2.508914,0.475000,00:02


epoch,train_loss,valid_loss,accuracy,time
0,1.814437,1.836110,0.729167,00:02
1,1.482514,0.775366,0.850000,00:01
2,1.148801,0.489849,0.895833,00:01
3,0.908958,0.433934,0.908333,00:01


CPU times: user 5.95 s, sys: 1.41 s, total: 7.37 s
Wall time: 10.2 s


In [19]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.487065,0.345709,0.904167,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.347123,0.276230,0.920833,00:01
1,0.284500,0.230856,0.929167,00:01
2,0.223709,0.224971,0.937500,00:01
3,0.169846,0.211307,0.937500,00:01


CPU times: user 5.54 s, sys: 1.12 s, total: 6.66 s
Wall time: 8.82 s


In [20]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj mom , xxmaj xxunk , xxmaj i 've been feeling really tired and weak lately , and xxmaj i 've had this cough that just wo n't go away . xxmaj it 's been hard for me to catch my breath , and my fever has been really high . xxmaj when i cough , xxmaj i 've been producing a lot of mucus .",Bronchial Asthma,Bronchial Asthma
1,"xxbos xxmaj i 've been having a tough time with this constant cough , difficulty breathing , and fatigue . xxmaj my fever is high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's all been rather xxunk and exhausting , and xxmaj i 'm feeling quite sick at the xxunk .",Bronchial Asthma,Bronchial Asthma
2,"xxbos xxmaj along with eyesight problems , increased appetite , a sore neck , melancholy , irritability , and regular headaches , i have also been experiencing acid reflux , indigestion , and frequent dizziness . xxmaj my regular activities have been made more tough by these symptoms , which have been causing me a great deal of distress .",Migraine,Migraine
3,"xxbos i keep sneezing , and my eyes do n't xxunk dripping . xxmaj it 's incredibly difficult for me to breathe because it feels like there is something trapped in my throat . i often feel exhausted , and lately , xxmaj i 've had a lot of phlegm . xxmaj moreover , my lymph nodes are enlarged .",Common Cold,Common Cold
4,"xxbos xxmaj my nose always feels stuffy and congested , and my eyes are always red and itching . i have a feeling of being unwell and fatigued , and i keep hacking up this gunk . i have a scratchy , irritated throat , and xxmaj i 've seen that my neck 's bumps are xxunk than usual .",Common Cold,Common Cold
5,"xxbos xxmaj i 've been having a lot of problems with itching , and it 's been accompanied by a rash that seems to be getting worse over time . xxmaj there are also some patches of skin that are different colors than the rest , and xxmaj i 've noticed some bumps that look like little nodes .",Fungal infection,Fungal infection
6,"xxbos xxmaj i 'm having a lot of problems breathing . xxmaj i 'm not feeling good , and xxmaj i 'm sweating a lot . i have a lot of mucous in my throat and my chest hurts . xxmaj my breathing is xxunk , and the phlegm xxmaj i 'm coughing up has an odd xxunk .",Pneumonia,Pneumonia
7,"xxbos i have a lot of difficulty breathing . i do n't feel well , and xxmaj i 'm perspiring a lot . xxmaj my chest hurts , and i have a lot of mucus in my throat . xxmaj i 'm having trouble breathing , and the phlegm xxmaj i 'm coughing up has a strange hue .",Pneumonia,Pneumonia
8,"xxbos xxmaj i 've been experiencing chills , feeling worn out , and i ca n't seem to get rid of this cough . xxmaj when i cough , my chest aches and my heart feels as like it is beating a million miles xxunk hour . xxmaj i 'm coughing up a nasty rust - colored phlegm .",Pneumonia,Pneumonia


### Inference

In [21]:
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")

('Jaundice',
 tensor(11),
 tensor([4.9424e-02, 9.3628e-03, 4.7467e-02, 2.2493e-01, 4.0574e-05, 5.2072e-03,
         1.5168e-03, 4.0896e-03, 4.1612e-03, 9.8209e-03, 1.8264e-03, 2.4204e-01,
         5.0225e-03, 1.0662e-01, 5.6693e-02, 5.1020e-03, 3.8718e-02, 1.7031e-03,
         4.7286e-02, 6.9751e-03, 1.5615e-03, 7.6947e-02, 7.1540e-03, 4.6334e-02]))

In [22]:
learn.predict("xxbos i have a lot of itching all over my skin , and sometimes it gets really red and bumpy . xxmaj there are also some patches that are different colors than the rest of my skin , and sometimes i get these little bumps that look like little balls . xxmaj it 's really annoying and itchy.Fungal ")

('Fungal infection',
 tensor(8),
 tensor([1.1666e-04, 9.7160e-04, 9.1755e-05, 2.8838e-05, 1.4735e-02, 1.2239e-05,
         1.2265e-04, 3.1399e-04, 9.6833e-01, 3.4538e-05, 2.4432e-05, 2.5759e-05,
         9.8103e-05, 1.6442e-05, 1.5211e-06, 2.2952e-03, 1.1423e-05, 1.8065e-04,
         3.1177e-04, 4.4286e-04, 1.0347e-02, 6.6468e-04, 3.7206e-04, 4.4988e-04]))

In [23]:
malaria_df = df[df['label'] == 'Malaria']
malaria_df.head()

,label,text
250,Malaria,"I've been experiencing severe itching, chills, vomiting, and a high fever. I'm also sweating a lot and have a headache. I feel nauseous and my muscles are really sore"
251,Malaria,"I've had a high temperature, vomiting, chills, and intense itching. I also have a headache and am perspiring a lot. I'm queasy, and my muscles hurt a lot."
252,Malaria,"I've had a high temperature, vomiting, chills, and intense itching. I also have a headache and am perspiring a lot. I'm queasy, and my muscles hurt a lot."
253,Malaria,"Strong itchiness, chills, nausea, and a high temperature have been plaguing me. Besides having a headache, I'm also perspiring a lot. My muscles are quite sore, and I feel sick."
254,Malaria,"Along with my intense scratching, chills, nausea, and high fever, I've also been feeling really unwell. Along with my headache and excessive sweating, My muscles hurt a lot, and I feel queasy."


### Trying to return probabilities

In [24]:
print(len(learn.dls.vocab))

2


In [25]:
#print(learn.dls.vocab)

In [26]:
def get_top_3_predictions(text, learn):
    # Get prediction and probabilities
    _, _, probs = learn.predict(text)
    
    # Get the disease labels vocabulary (second list in vocab)
    disease_vocab = learn.dls.vocab[1]  # Access the disease labels
    
    # Get number of classes
    n_classes = len(disease_vocab)
    
    # Get indices of top 3 (or fewer) probabilities
    n_preds = min(3, n_classes)
    top_k_indices = probs.argsort(descending=True)[:n_preds]
    
    # Get the actual labels and their probabilities
    predictions = []
    for idx in top_k_indices:
        label = disease_vocab[int(idx)]  # Use disease_vocab instead of vocab
        probability = float(probs[idx])
        predictions.append((label, f"{probability:.3f}"))
    
    return predictions

def predict_batch_top_3(texts, learn):
    predictions = []
    for text in texts:
        preds = get_top_3_predictions(text, learn)
        predictions.append(preds)
    return predictions

# Test with a single prediction first
test_text = "severe headache with light sensitivity"
single_pred = get_top_3_predictions(test_text, learn)
print("\nSingle prediction test:")
for i, (disease, prob) in enumerate(single_pred, 1):
    print(f"{i}. {disease}: {prob}")

# Then test batch predictions
test_texts = ["severe headache with light sensitivity", 
              "itchy red rash with scaling",
              "frequent urination and excessive thirst"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Single prediction test:
1. Migraine: 0.902
2. urinary tract infection: 0.023
3. diabetes: 0.018



Batch predictions:

Text 1:
1. Migraine: 0.902
2. urinary tract infection: 0.023
3. diabetes: 0.018

Text 2:
1. Migraine: 0.962
2. Psoriasis: 0.015
3. urinary tract infection: 0.008

Text 3:
1. Migraine: 0.958
2. urinary tract infection: 0.029
3. Acne: 0.004


In [27]:
test_texts = ["severe headache with light sensitivity"]
_, _, probs = learn.predict(test_texts)

In [ ]:
probs

In [ ]:
# Then test batch predictions
test_texts = ["severe headache with light sensitivity"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")

In [ ]:
# Test with a single prediction first
test_text = "i have a severe running stomach, cold chills, fever, excessive sweating, fatigue and joint pain"

single_pred = get_top_3_predictions(test_text, learn)
print("\nSingle prediction test:")
for i, (disease, prob) in enumerate(single_pred, 1):
    print(f"{i}. {disease}: {prob}")


## Using the data block API

In [ ]:
#dis = DataBlock(
    #blocks=(TextBlock.from_df('text', seq_len=72), CategoryBlock),
    #get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

In [28]:
dis_x = DataBlock(
    blocks=(TextBlock.from_df('text'), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('label'),
    splitter=RandomSplitter(valid_pct=0.2, seed=42)
)

In [29]:
dls = dis_x.dataloaders(df, bs=64)
dls.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma


# ULMFiT approach

Our initial pre-trained model used above was initially trained on Wikipedia on the task of guessing the next word. We then fine-tuned this model for our disease classification task based on symptoms.

But the Wikipedia English might differ from medical jargon, so to further improve our model, We can take this a step further by fitting this pre-trained model on medical corpus and using that as a the base for our classifier.

In [42]:
!ls /kaggle/input/symptom-dataset-synthetic

symptom_synth.csv


In [46]:
path_lm = Path('/kaggle/input/symptom-dataset-synthetic')
path_lm

Path('/kaggle/input/symptom-dataset-synthetic')

In [47]:
path_lm

Path('/kaggle/input/symptom-dataset-synthetic')

In [69]:
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
symptom_df = pd.read_csv(path_lm/'symptom_synth.csv')
symptom_df.head()

,text
0,"I't feelve noticed and the l sinuittal have bey on my cal. I'm st coughingtoider,tous my back and my eyes are starting and have a temperature. A't to goike anything. Additionally, chest a headache, and my neck hurts and diarrhs when I can't seem to eu of my skin. I can'tn anything and have been experiencing un, irritability, and my appetite and. In addition, and I've also been feeling stomach pain that cered to be and cest. I've had some {"
1,"I have been experiencing severeular, a cough and headache. It is causing me to bely a headache of mucous. I regularly've been suffering from nausea, and a a headache."
2,"I have been experiencing severe itching, chills, inc and constipation, and breathing. I'm feeling exhausted by a scratchyor lot."
3,"I have been experiencing a high fever, and headache, dizziness, a lot of thick, and the psesees that's some rtids of. Sometimes sores is b"
4,"Migraine muscles have a lot of taste for, I't go and feel fatigued. When and I have rashes on my skin mr. I have a fever, I've been suffering and a lot of troublet sting."


In [70]:
symptom_df['text'].nunique()

8766

In [55]:
df['text'].nunique()

1153

In [68]:
df.columns

Index(['label', 'text'], dtype='object')

In [72]:
symptom_df.columns

Index(['text'], dtype='object')

In [73]:
symptom_df['text'].unique()

array(["I't feelve noticed and the l sinuittal have bey on my cal. I'm st coughingtoider,tous my back and my eyes are starting and have a temperature. A't to goike anything. Additionally, chest a headache, and my neck hurts and diarrhs when I can't seem to eu of my skin. I can'tn anything and have been experiencing un, irritability, and my appetite and. In addition, and I've also been feeling stomach pain that cered to be and cest. I've had some {",
       "I have been experiencing severeular, a cough and headache. It is causing me to bely a headache of mucous. I regularly've been suffering from nausea, and a  a headache.",
       "I have been experiencing severe itching, chills, inc and constipation, and breathing. I'm feeling exhausted by a scratchyor lot.",
       ..., "My tt to do'mble and limbises in my muscles recently.",
       "I'm having a stiff cough and back, especially I have developed weight, and dizziness.",
       "Along itching, and I've also been experiencing high feve

In [36]:
symptom_df['text'].unique()

array(['I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.',
       'My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.',
       'I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints.',
       ...,
       "I regularly experience these intense urges and the want to urinate. I frequently feel drowsy and lost. I've also significantly lost my vision.",
       'I have trouble breathing, especially outside. I start to feel hot and start to sweat. I frequently have urinary tract infections and yeast infections.',
       "I constantly sneeze and have a dry cough. My infections don't seem to be healing, and I have palpitations. My throat does ache occasionally, but it usually gets better."],
      dtype=object)

In [74]:
dls_lm = TextDataLoaders.from_df(symptom_df, path=path_lm, is_lm=True, valid_pct=0.2)
#dls_lm = TextDataLoaders.from_folder(path=path_lm, is_lm=True, valid_pct=0.1)

In [75]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos i have been experiencing a lot of pain and a fever . xxmaj i 've also experienc a sore skin , and the on my xxunk on my back . xxmaj i 've been having stomach pain , xxbos xxmaj i 'm having trouble xxunk , and intense , and am uns been feeling extremely uncomfortable to ting my skin . xxbos i have a severe cough , red and it","i have been experiencing a lot of pain and a fever . xxmaj i 've also experienc a sore skin , and the on my xxunk on my back . xxmaj i 've been having stomach pain , xxbos xxmaj i 'm having trouble xxunk , and intense , and am uns been feeling extremely uncomfortable to ting my skin . xxbos i have a severe cough , red and it difficult"
1,"sour and muscle . xxbos xxmaj varicose xxbos i have been experiencing a , chills , chills , chills , a sore neck , and it are painful uncomfortableing . xxbos xxmaj i 've been having a lot of trouble my skin . xxmaj sinces are conp - filled . xxmaj i 'm sweating a lot of discomfort , and headache . i occasionally have been having this in breathing . xxbos","and muscle . xxbos xxmaj varicose xxbos i have been experiencing a , chills , chills , chills , a sore neck , and it are painful uncomfortableing . xxbos xxmaj i 've been having a lot of trouble my skin . xxmaj sinces are conp - filled . xxmaj i 'm sweating a lot of discomfort , and headache . i occasionally have been having this in breathing . xxbos xxmaj"
2,"ca n't go xxunk and my neck hurts . xxmaj my s are a sour fever xxbos xxmaj i 've had a temperature , and a lot . xxmaj i 'm coughing up a lot . xxmaj additionally , i have been a terrible never that s my neck and xxmaj i 've had a lot of mucus that on my skin is swollen . xxbos } { xxbos xxmaj my skin","n't go xxunk and my neck hurts . xxmaj my s are a sour fever xxbos xxmaj i 've had a temperature , and a lot . xxmaj i 'm coughing up a lot . xxmaj additionally , i have been a terrible never that s my neck and xxmaj i 've had a lot of mucus that on my skin is swollen . xxbos } { xxbos xxmaj my skin has"
3,", and xxmaj i 've been experiencingular it 's me to to move without about a lot . xxbos i have been experiencing intense itching in back pain , a temperature , dep , and dizziness . i have been having a high fever and experienc xxbos xxmaj the xxmaj hemorrhoids a headache , which , headaches , my muscles skin , a headache a ch and xxunk . i am having","and xxmaj i 've been experiencingular it 's me to to move without about a lot . xxbos i have been experiencing intense itching in back pain , a temperature , dep , and dizziness . i have been having a high fever and experienc xxbos xxmaj the xxmaj hemorrhoids a headache , which , headaches , my muscles skin , a headache a ch and xxunk . i am having trouble"
4,"it and have a mild fever . i do xxunk and have frequent m and feeling of skin . xxmaj it alsos have been difficult to deal of stiff of the time of breath . xxmaj the mucus urine , xxmaj i 'm sweating and my skin . xxmaj i 'm really ce up a lot of discomfort in my heal and my appetite . xxmaj it 's been feeling so really","and have a mild fever . i do xxunk and have frequent m and feeling of skin . xxmaj it alsos have been difficult to deal of stiff of the time of breath . xxmaj the mucus urine , xxmaj i 'm sweating and my skin . xxmaj i 'm really ce up a lot of discomfort in my heal and my appetite . xxmaj it 's been feeling so really to"


In [76]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path_lm, wd=0.1).to_fp16()

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [77]:
learn.fit_one_cycle(1, 1e-2)

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.530691,2.989921,0.351377,19.884119,00:12


In [15]:
#learn.save('/kaggle/working/1epoch')

In [78]:
import os

# Create a directory to save the model
os.makedirs('/kaggle/working/models', exist_ok=True)

# Set the model directory for the learner
learn.model_dir = '/kaggle/working/models'

# Now save the model
learn.save('1epoch')

Path('/kaggle/working/models/1epoch.pth')

In [79]:
learn = learn.load('1epoch')

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)


In [80]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.000735,2.801724,0.373678,16.473021,00:14
1,2.854630,2.701609,0.379880,14.903694,00:14
2,2.751140,2.628494,0.387137,13.852896,00:14
3,2.678902,2.603837,0.387136,13.515498,00:14
4,2.633186,2.590053,0.387315,13.330482,00:14
5,2.596886,2.574984,0.389829,13.131104,00:14
6,2.574257,2.572211,0.389589,13.094748,00:14
7,2.562042,2.568398,0.389957,13.044916,00:14
8,2.544447,2.566116,0.390426,13.015180,00:14
9,2.533921,2.565554,0.390947,13.007861,00:14


In [81]:
# Now save the model
learn.save_encoder('finetuned')

In [20]:
#learn.save_encoder('finetuned')

In [82]:
TEXT = "I have running nose, stomach and joint pains"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

In [83]:
print("\n".join(preds))

i have xxunk nose , stomach and xxunk pains on my nose . I 've been having a lot of trouble of the time , and I 've been having a lot of thick , and I 've had a lot of and feeling really weak
i have xxunk nose , stomach and xxunk pains , and i have a headache and have a lot . My urine and are swollen , i have lost weight and am experiencing stomach pain . My anus is really uncomfortable , and it 's hard for


In [90]:
df = pd.read_csv('/kaggle/input/s2d-synth/s2d_synth.csv')
df.head()

,label,text
0,Common Cold,"I't feelve noticed and the l sinuittal have bey on my cal. I'm st coughingtoider,tous my back and my eyes are starting and have a temperature. A't to goike anything. Additionally, chest a headache, and my neck hurts and diarrhs when I can't seem to eu of my skin. I can'tn anything and have been experiencing un, irritability, and my appetite and. In addition, and I've also been feeling stomach pain that cered to be and cest. I've had some {"
1,Pneumonia,"I have been experiencing severeular, a cough and headache. It is causing me to bely a headache of mucous. I regularly've been suffering from nausea, and a a headache."
2,Psoriasis,"I have been experiencing severe itching, chills, inc and constipation, and breathing. I'm feeling exhausted by a scratchyor lot."
3,Dengue,"I have been experiencing a high fever, and headache, dizziness, a lot of thick, and the psesees that's some rtids of. Sometimes sores is b"
4,Impetigo,"Migraine muscles have a lot of taste for, I't go and feel fatigued. When and I have rashes on my skin mr. I have a fever, I've been suffering and a lot of troublet sting."


In [91]:
dls_clas = TextDataLoaders.from_df(df, path=path, text_vocab=dls_lm.vocab)

/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


In [92]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [93]:
from pathlib import Path
learn.path = Path('/kaggle/working')

In [94]:
learn = learn.load_encoder('finetuned')

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(join_path_file(file,self.path/self.model_

In [95]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time


KeyError: Caught KeyError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py", line 263, in encodes
    return TensorCategory(self.vocab.o2i[o])
KeyError: nan

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 42, in fetch
    data = next(self.dataset_iter)
  File "/opt/conda/lib/python3.10/site-packages/fastai/data/load.py", line 140, in create_batches
    yield from map(self.do_batch, self.chunkify(res))
  File "/opt/conda/lib/python3.10/site-packages/fastcore/basics.py", line 245, in chunked
    res = list(itertools.islice(it, chunk_sz))
  File "/opt/conda/lib/python3.10

In [31]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time


/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels 

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/fastai/data/transforms.py", line 263, in encodes
    return TensorCategory(self.vocab.o2i[o])
KeyError: "My persistent cough, breathing problems, and exhaustion have been making things difficult for me. I've been producing copious amounts of thick, mucoid sputum while also having a high fever. I'm currently feeling fairly ill because everything has been so tiring and exhausting."

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 42, in fetch
    data = next(self.dataset_iter)
  File "/opt/conda/lib/python3.10/site-packages

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")